# 3. 뱀게임과 유전 알고리즘 결합

이 노트북에서는 신경망과 유전 알고리즘을 결합하여 뱀게임을 플레이하는 AI를 만들어보겠습니다.

## 목차
1. [신경망 구현](#1-신경망-구현)
2. [게임 상태를 신경망 입력으로 변환](#2-게임-상태를-신경망-입력으로-변환)
3. [유전 알고리즘으로 신경망 훈련](#3-유전-알고리즘으로-신경망-훈련)
4. [학습 과정 시각화](#4-학습-과정-시각화)
5. [최고 AI와 사람 비교](#5-최고-ai와-사람-비교)

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import sys
sys.path.append('..')

# 이전 노트북의 뱀게임 클래스 import
from notebooks.snake_game_basics import SimpleSnakeGame

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 6)

print("라이브러리 로드 완료!")


## 1. 신경망 구현

먼저 간단한 피드포워드 신경망을 구현하겠습니다.


In [ ]:
class SimpleNeuralNetwork:
    """간단한 피드포워드 신경망"""
    
    def __init__(self, layers):
        """
        layers: 각 층의 뉴런 수를 나타내는 리스트
        예: [11, 20, 20, 4] -> 입력 11개, 은닉층 20개씩 2개, 출력 4개
        """
        self.layers = layers
        self.weights = []
        self.biases = []
        
        # 가중치와 편향 초기화
        for i in range(len(layers) - 1):
            # He 초기화 사용
            weight = np.random.randn(layers[i], layers[i+1]) * np.sqrt(2.0 / layers[i])
            bias = np.zeros((1, layers[i+1]))
            
            self.weights.append(weight)
            self.biases.append(bias)
    
    def relu(self, x):
        """ReLU 활성화 함수"""
        return np.maximum(0, x)
    
    def softmax(self, x):
        """Softmax 활성화 함수"""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, x):
        """순전파"""
        if len(x.shape) == 1:
            x = x.reshape(1, -1)
        
        activation = x
        
        # 은닉층들 (ReLU 사용)
        for i in range(len(self.weights) - 1):
            z = np.dot(activation, self.weights[i]) + self.biases[i]
            activation = self.relu(z)
        
        # 출력층 (Softmax 사용)
        z = np.dot(activation, self.weights[-1]) + self.biases[-1]
        output = self.softmax(z)
        
        return output
    
    def get_weights_as_vector(self):
        """모든 가중치와 편향을 1차원 벡터로 변환"""
        vector = []
        for w, b in zip(self.weights, self.biases):
            vector.extend(w.flatten())
            vector.extend(b.flatten())
        return np.array(vector)
    
    def set_weights_from_vector(self, vector):
        """1차원 벡터를 가중치와 편향으로 변환"""
        idx = 0
        for i in range(len(self.weights)):
            # 가중치 설정
            w_size = self.weights[i].size
            self.weights[i] = vector[idx:idx+w_size].reshape(self.weights[i].shape)
            idx += w_size
            
            # 편향 설정
            b_size = self.biases[i].size
            self.biases[i] = vector[idx:idx+b_size].reshape(self.biases[i].shape)
            idx += b_size
    
    def copy(self):
        """신경망 복사"""
        new_nn = SimpleNeuralNetwork(self.layers)
        new_nn.set_weights_from_vector(self.get_weights_as_vector())
        return new_nn

# 신경망 테스트
nn = SimpleNeuralNetwork([11, 16, 16, 4])  # 입력 11개, 출력 4개 (상,우,하,좌)
test_input = np.random.rand(11)
output = nn.forward(test_input)

print(f"신경망 구조: {nn.layers}")
print(f"테스트 입력 크기: {test_input.shape}")
print(f"출력 확률: {output[0]}")
print(f"선택된 행동: {np.argmax(output[0])}")
print(f"전체 파라미터 수: {len(nn.get_weights_as_vector())}")
